Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [32]:
import tensorflow as tf # Deep Learning Tool
from sklearn.model_selection import train_test_split # For splitting the data into train and validation set
from sklearn.metrics import f1_score

In [33]:
train = pd.read_csv('eye_gender_data/Training_set.csv')

Converting male-female to 0-1

In [34]:
def male_female(x):
    if x == 'male':
        return 0
    else:
        return 1

Creating Training Dataset

In [35]:
labels = pd.read_csv("eye_gender_data/Training_set.csv") # loading the labels
file_paths = [[fname, 'eye_gender_data/train/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')

x_train = [] # initialize an empty numpy array
image_size = 53 # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):

    img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale

    new_img_array = cv2.resize(img_array,( image_size, image_size)) # resizing the image array
    x_train.append(new_img_array)
x_train = np.asarray(x_train, dtype=int)
x_train = x_train.reshape(9220,53,53,1)

In [36]:
train['label'] = train.apply(lambda x: male_female(x.label),axis =1 )
y_train = train['label'].to_numpy()

Data Preprocessing

In [37]:
x_train = x_train/255
x_train = 2*x_train - 1

Defining the Model

In [53]:
# define input shape
INPUT_SHAPE = (53, 53, 1)

# define sequential model
model = tf.keras.models.Sequential()
# define conv-pool layers - set 1
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1),
activation='relu', padding='valid', input_shape=INPUT_SHAPE))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
# define conv-pool layers - set 2
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1),
activation='relu', padding='valid', input_shape=INPUT_SHAPE))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# add flatten layer
model.add(tf.keras.layers.Flatten())

# add dense layers with some dropout
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.3))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dropout(rate=0.3))

# add output layer
model.add(tf.keras.layers.Dense(2, activation='softmax'))

# compile model
model.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])

# view model layers
# model.summary()

Training the model

In [54]:
EPOCHS = 5
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2,
restore_best_weights=True,
verbose=1)

history = model.fit(x_train,
y_train,
batch_size=32,
callbacks=[es_callback],
validation_split=0.1, epochs=EPOCHS,
verbose=1)

Epoch 1/5
260/260 [==============================] - 9s 35ms/step - loss: 0.4745 - accuracy: 0.7674 - val_loss: 0.3576 - val_accuracy: 0.8471
Epoch 2/5
260/260 [==============================] - 9s 35ms/step - loss: 0.3226 - accuracy: 0.8594 - val_loss: 0.3105 - val_accuracy: 0.8688
Epoch 3/5
260/260 [==============================] - 7s 27ms/step - loss: 0.2523 - accuracy: 0.8976 - val_loss: 0.2950 - val_accuracy: 0.8731
Epoch 4/5
260/260 [==============================] - 7s 27ms/step - loss: 0.2113 - accuracy: 0.9167 - val_loss: 0.2206 - val_accuracy: 0.9089
Epoch 5/5
260/260 [==============================] - 9s 33ms/step - loss: 0.1731 - accuracy: 0.9323 - val_loss: 0.2510 - val_accuracy: 0.8915


Making Test Dataset

In [55]:
labels = pd.read_csv("eye_gender_data/Testing_set.csv") # loading the labels
file_paths = [[fname, 'eye_gender_data/test/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_data = pd.merge(images, labels, how = 'inner', on = 'filename')

x_test = [] # initialize an empty numpy array
image_size = 53 # image size taken is 100 here. one can take other size too
for i in range(len(test_data)):

    img_array = cv2.imread(test_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale

    new_img_array = cv2.resize(img_array,( image_size, image_size)) # resizing the image array
    x_test.append(new_img_array)
x_test = np.asarray(x_test, dtype=int)
x_test = x_test.reshape(2305,53,53,1)

Data Preprocessing

In [56]:
x_test = x_test/255
x_test = 2*x_test - 1

Running the Model

In [57]:
predictions = model.predict(x_test)
prediction_labels = np.argmax(predictions, axis=1)
pred = list(prediction_labels)

Converting from 0-1 to male-female

In [75]:
for i in range(len(pred)):
    if pred[i] == 0:
        pred[i] = 'male'
    else:
        pred[i] = 'female'

In [76]:
res = pd.DataFrame({'label': pred}) # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)